In [ ]:
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

In [ ]:
# Load filter functions
filtershape = fits.open('/Users/dahlek/Desktop/Filter_Transmission_Functions/filtershape_march2020.fits')

In [ ]:
# Plot 2D filter functions
plt.matshow(filtershape[0].data)
plt.colorbar()
plt.show()

In [ ]:
# Plot some filters for whichever wavelengths you'd like to examine

first_wl = filtershape[0].header['lambdlow']
wl1 = 480
wl2 = 500
wl3 = 530
wl4 = 550 
wl5 = 580
wl6 = 650

index1 = wl1-first_wl
index2 = wl2-first_wl
index3 = wl3-first_wl
index4 = wl4-first_wl
index5 = wl5-first_wl
index6 = wl6-first_wl

mine_wl1 = np.linspace(wl1-25,wl1+25,101)
mine_wl2 = np.linspace(wl2-25,wl2+25,101)
mine_wl3 = np.linspace(wl3-25,wl3+25,101)
mine_wl4 = np.linspace(wl4-25,wl4+25,101)
mine_wl5 = np.linspace(wl5-25,wl5+25,101)
mine_wl6 = np.linspace(wl6-25,wl6+25,101)

plt.plot(mine_wl1,filtershape[0].data[:,index1])
plt.plot(mine_wl2,filtershape[0].data[:,index2])
plt.plot(mine_wl3,filtershape[0].data[:,index3])
plt.plot(mine_wl4,filtershape[0].data[:,index4])
plt.plot(mine_wl5,filtershape[0].data[:,index5])
plt.plot(mine_wl6,filtershape[0].data[:,index6])


plt.legend([str(wl1),str(wl2),str(wl3),str(wl4),str(wl5),str(wl6)])

plt.show()

In [ ]:
# Loop through wavelengths of interest (np.arange(0,241*2+1,2)), calculate FWHM at each wavelength

first_wl = filtershape[0].header['lambdlow']
ff = filtershape[0].data
fwhm_list = []
wavelength = []

for index in np.arange(0,241*2,2):
    wl = first_wl+index
    wavelength.append(wl)
    print wl, index
    filter_shape = ff[:,index]
    filter_wavelengths = np.linspace(wl-25,wl+25,filtershape[0].header['naxis2'])
    
    # Find minima on either side of central peak
    center_peak_index = len(ff[:,index])/2

    a = ff[:,index]
    minima_span = 1
    boo = np.r_[True, a[minima_span:] < a[:-minima_span]] & np.r_[a[:-minima_span] < a[minima_span:], True]
    
    for x in list(reversed(range(center_peak_index))):
        if boo[x] == True:
            min1_index = x
            break
    for x in range(center_peak_index,len(boo)):
        if boo[x] == True:
            min2_index = x # the index of the first minima on the right side of the main lobe. ff[x,index] is its value
            break # stop the loop        

    wl_sidelobes = []
    transmission_sidelobes = []

    wl_center = []
    transmission_center = []

    #make an array with the wavelengths and transmission at all the points outside of the main lobe
    for y in range(0,len(filter_wavelengths)):
        if y < min1_index or y > min2_index:
            wl_sidelobes.append(filter_wavelengths[y])
            transmission_sidelobes.append(ff[y,index])
        # Adding more indeces to get on either side of FWHM interpolation range
        extra = 4 # for my filters
        #extra = 0 # for pauls
        if y > min1_index-extra and y < min2_index+extra:
            wl_center.append(filter_wavelengths[y])
            transmission_center.append(ff[y,index])
    
    
    # Find FWHM
    
    # find index of center of center lobe
    val, idx = max((val, idx) for (idx, val) in enumerate(transmission_center))
    
    func1 = interp1d(transmission_center[0:idx],wl_center[0:idx]) # left side of maximum
    func2 = interp1d(transmission_center[idx:],wl_center[idx:]) # right side of maximum
    FWHM = round(func2(0.5) - func1(0.5),3)
    plt.plot(wl_center,transmission_center,'.')
    plt.plot(wl_sidelobes,transmission_sidelobes,'.')
    plt.plot(func1(0.5),0.5,'ro')
    plt.plot(func2(0.5),0.5,'ro')
    plt.show()
    print FWHM
    fwhm_list.append(FWHM)
    
    

In [ ]:
# Plot filter FWHM as a function of wavelength
plt.plot(wavelength,fwhm_list,'.')
plt.xlabel('Wavelength (nm)')
plt.ylabel('FWHM of center filter lobe (nm)')
plt.show()

In [ ]:
%matplotlib qt5

In [ ]:
# find lambda/d lambda for each wavelength
spec_res = []
for i in range(0,len(wavelength)):
    spec_res.append(wavelength[i]/fwhm_list[i])

In [ ]:
# Plot lambda/d lambda 
plt.plot(wavelength,spec_res,'.')
plt.xlabel('Wavelength (nm)')
plt.ylabel('$\lambda$/d$\lambda$')
plt.show()

In [ ]:
# Return the average spectral resolution
np.mean(spec_res)

In [ ]:
# Return the average FWHM
np.mean(fwhm_list)